In [15]:
import cv2 as cv
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
# Function to load image data
def load_data(dir_list, image_size):
    data = []
    label = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in os.listdir(directory):
            image = cv.imread(os.path.join(directory, filename), 0)
            image = cv.resize(image, dsize=(image_width, image_height), interpolation=cv.INTER_CUBIC)
            image = image / 255.0
            image = image.astype(np.float32)
            data.append(image.flatten())  # Flatten the image to use with SVM
            if directory == 'yes':
                label.append(1)  # Tumor
            else:
                label.append(0)  # No tumor
    data_array = np.array(data)
    label = np.array(label) 
    return data_array, label

In [17]:
# Load data
x, y = load_data(['yes', 'no'], (256, 256))


In [18]:
# Split data
def split_data(test_size=0.2, log=True):
    x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=test_size, shuffle=True)
    if log:
        print('Input Shape:')
        print(x_train.shape, y_train.shape)
        print(x_test_val.shape, y_test_val.shape)
    x_test = x_test_val
    x_val = x_test_val
    y_test = y_test_val
    y_val = y_test_val
    if log:
        print('Output Shape:')
        print(x_train.shape, y_train.shape)
        print(x_test.shape, y_test.shape)
        print(x_val.shape, y_val.shape)
    return x_train, x_test, y_train, y_test, x_val, y_val

x_train, x_test, y_train, y_test, x_val, y_val = split_data()
validation_dataset = (x_val, y_val)


Input Shape:
(2400, 65536) (2400,)
(600, 65536) (600,)
Output Shape:
(2400, 65536) (2400,)
(600, 65536) (600,)
(600, 65536) (600,)


In [19]:
# SVM Model
class SVM:
    def __init__(self, lr=0.001, epochs=100):
        self.lr = lr
        self.epochs = epochs
        self.weights = None  # Initialize weights
        self.bias = None  # Initialize bias
        
    def train(self, X, y):
        self.X = X
        self.y = y
        self.n_samples, self.n_features = X.shape
        
        # Initialize weights and bias
        self.weights = np.zeros(self.n_features) if self.weights is None else self.weights
        self.bias = 0 if self.bias is None else self.bias
        
        # Gradient descent
        for _ in range(self.epochs):
            for idx, x_i in enumerate(self.X):
                condition = self.y[idx] * (np.dot(x_i, self.weights) - self.bias) >= 1
                if condition:
                    self.weights -= self.lr * (2 * 1 / self.epochs * self.weights)
                else:
                    self.weights -= self.lr * (2 * 1 / self.epochs * self.weights - np.dot(x_i, self.y[idx]))
                    self.bias -= self.lr * self.y[idx]
    
    def predict(self, X):
        return np.sign(np.dot(X, self.weights) - self.bias)


In [20]:
# Train SVM Model
svm_model = SVM()
svm_model.train(x_train, y_train)

In [21]:
# Make predictions
y_train_pred = svm_model.predict(x_train)
y_test_pred = svm_model.predict(x_test)

In [22]:

# Calculate accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [23]:

print("SVM Train Accuracy:", train_accuracy)
print("SVM Test Accuracy:", test_accuracy)

SVM Train Accuracy: 0.5095833333333334
SVM Test Accuracy: 0.46166666666666667


In [24]:
import pickle

# Save SVM model
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

# Save the trained SVM model
save_model(svm_model, 'svm_model.pkl')